## Lista de Exercícios #12: Redes Neurais Convolucionais (CNN)

**Aluno:** Samuel Horta de Faria

**Matrícula:** 801528

**Curso:** Ciência da Computação
**Disciplina:** Inteligência Artificial

**Profª:** Cristiane Neri Nobre

Este notebook implementa uma Rede Neural Convolucional (CNN) para resolver o problema de classificação de imagens da competição 'Dogs vs. Cats' do Kaggle. O objetivo é construir um modelo capaz de diferenciar imagens de cachorros e gatos. As etapas incluem a preparação dos dados, a construção da arquitetura da CNN, o treinamento do modelo e a geração de um arquivo de submissão.

### 0. Configurações Iniciais e Imports

Nesta seção, importamos todas as bibliotecas necessárias para o projeto. Utilizaremos `pandas` para manipulação de dados, `numpy` para operações numéricas, `matplotlib` para visualização, e `tensorflow.keras` para construir e treinar nossa CNN.

In [ ]:
# Imports para manipulação de dados e sistema operacional
import numpy as np
import pandas as pd
import os
import random

# Imports para visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

# Imports do TensorFlow e Keras para a construção da CNN
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Import para divisão de dados
from sklearn.model_selection import train_test_split

# Configuração para exibir gráficos no notebook
%matplotlib inline

### 1. Definição de Constantes e Caminhos

Definir constantes no início do código é uma boa prática, pois facilita a manutenção e a alteração de parâmetros importantes, como o tamanho das imagens e os caminhos para os diretórios de dados.

In [ ]:
# --- ATENÇÃO: Altere os caminhos para os diretórios do seu computador ---
TRAIN_DIR = 'dogs-vs-cats/train/'
TEST_DIR = 'dogs-vs-cats/test1/'

# Constantes para o modelo
IMAGE_WIDTH = 128
IMAGE_HEIGHT = 128
IMAGE_SIZE = (IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS = 3 # 3 para imagens coloridas (RGB)
BATCH_SIZE = 32 # Número de imagens processadas por lote

### 2. Preparação dos Dados de Treino

Os dados de treino estão em uma única pasta, com os nomes dos arquivos indicando a classe (`cat.x.jpg` ou `dog.x.jpg`). Vamos ler todos os nomes de arquivos, extrair suas classes e organizar essas informações em um DataFrame do pandas para facilitar a manipulação.

In [ ]:
# Lista para armazenar nomes de arquivos e suas categorias
filenames = os.listdir(TRAIN_DIR)
categories = []

# Loop para extrair a categoria do nome do arquivo
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append('dog') # Usando strings para facilitar a divisão
    else:
        categories.append('cat')

# Criação do DataFrame
train_df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

print("---| DataFrame de Treino Criado |---")
print(f"Total de imagens de treino: {train_df.shape[0]}")
print(train_df.head())

print("\n---| Distribuição das Classes |---")
print(train_df['category'].value_counts())

### 3. Divisão dos Dados e Geração de Lotes (Batches)

Para avaliar o modelo de forma justa, dividimos o conjunto de treino em dois: um para treinamento (90%) e outro para validação (10%). A validação nos ajuda a monitorar se o modelo está sofrendo de overfitting (decorando os dados de treino em vez de aprender a generalizar).

Usaremos `ImageDataGenerator` do Keras para:
1.  **Aumento de Dados (Data Augmentation):** Criar variações das imagens de treino (rotações, zooms, etc.) para tornar o modelo mais robusto.
2.  **Geração de Lotes:** Carregar as imagens da memória de forma eficiente em lotes, redimensionando-as e normalizando os valores dos pixels (dividindo por 255).

In [ ]:
# Divisão do DataFrame em treino e validação
train_df, validate_df = train_test_split(train_df, test_size=0.1, random_state=42, stratify=train_df['category'])
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

print("---| Tamanho dos Conjuntos Após a Divisão |---")
print(f"Treino: {train_df.shape[0]} imagens")
print(f"Validação: {validate_df.shape[0]} imagens")

# Gerador para dados de treino com Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255.,
    rotation_range=15,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

# Gerador para dados de validação (apenas normalização)
validation_datagen = ImageDataGenerator(rescale=1./255.)

# Criando os geradores que lerão as imagens a partir do DataFrame
train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    TRAIN_DIR,
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='binary', # Classificação binária (gato/cachorro)
    batch_size=BATCH_SIZE
)

validation_generator = validation_datagen.flow_from_dataframe(
    validate_df,
    TRAIN_DIR,
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='binary',
    batch_size=BATCH_SIZE
)

### 4. Construção do Modelo CNN

Agora, definimos a arquitetura da nossa CNN. O modelo será sequencial e composto por:
- **Camadas Convolucionais (`Conv2D`):** Responsáveis por extrair características das imagens (bordas, texturas, formas).
- **Camadas de Pooling (`MaxPooling2D`):** Reduzem a dimensionalidade dos mapas de características, mantendo as informações mais importantes.
- **Camada Flatten:** Transforma a matriz 2D de características em um vetor 1D para ser processado pela parte final da rede.
- **Camadas Densas (`Dense`):** Camadas totalmente conectadas que realizam a classificação com base nas características extraídas.
- **Camada de Dropout:** Técnica de regularização para prevenir overfitting, "desligando" aleatoriamente alguns neurônios durante o treino.
- **Funções de Ativação:** `relu` para as camadas ocultas e `sigmoid` para a camada de saída, pois temos uma classificação binária.

In [ ]:
model = Sequential()

# Bloco 1
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Bloco 2
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Bloco 3
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Camada de achatamento e camadas densas
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5)) # Dropout para regularização
model.add(Dense(1, activation='sigmoid')) # Saída com sigmoid para classificação binária

# Compilação do modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("---| Resumo da Arquitetura do Modelo |---")
model.summary()

### 5. Treinamento do Modelo

Com o modelo e os geradores de dados prontos, iniciamos o treinamento usando o método `fit()`. O treinamento ocorrerá por um número definido de `epochs` (épocas). Em cada época, o modelo verá todas as imagens do conjunto de treino.

*Nota: O treinamento pode demorar dependendo do hardware (CPU vs GPU).* 

In [ ]:
EPOCHS = 15 # Um número razoável para um bom começo. Pode ser aumentado para melhores resultados.

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validate_df.shape[0] // BATCH_SIZE,
    steps_per_epoch=train_df.shape[0] // BATCH_SIZE
)

### 6. Avaliação do Modelo

Após o treinamento, é crucial avaliar o desempenho. Plotamos a acurácia e a perda (loss) do treino e da validação ao longo das épocas. Isso nos permite visualizar o aprendizado e identificar sinais de overfitting (quando a acurácia de treino continua subindo, mas a de validação estagna ou cai).

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))

# Gráfico da Acurácia
ax1.plot(history.history['accuracy'], color='b', label="Acurácia de Treino")
ax1.plot(history.history['val_accuracy'], color='r', label="Acurácia de Validação")
ax1.set_xticks(np.arange(0, EPOCHS, 1))
ax1.set_ylabel('Acurácia')
ax1.set_title('Acurácia por Época')
ax1.legend()

# Gráfico da Perda
ax2.plot(history.history['loss'], color='b', label="Perda de Treino")
ax2.plot(history.history['val_loss'], color='r', label="Perda de Validação")
ax2.set_xticks(np.arange(0, EPOCHS, 1))
ax2.set_ylabel('Perda')
ax2.set_xlabel('Época')
ax2.set_title('Perda por Época')
ax2.legend()

plt.show()

### 7. Preparação dos Dados de Teste e Geração da Submissão

Finalmente, usamos o modelo treinado para prever as classes das imagens no conjunto de teste. O processo é semelhante ao da validação: criamos um DataFrame para os arquivos de teste e um gerador para carregá-los e normalizá-los. As previsões do modelo (probabilidades entre 0 e 1) são então convertidas para as classes finais (0 para 'gato', 1 para 'cachorro') e formatadas em um arquivo .csv para submissão no Kaggle.

In [ ]:
# Preparando o DataFrame de teste
test_filenames = os.listdir(TEST_DIR)
test_df = pd.DataFrame({
    'filename': test_filenames
})
nb_samples = test_df.shape[0]

# Gerador para os dados de teste
test_gen = ImageDataGenerator(rescale=1./255.)
test_generator = test_gen.flow_from_dataframe(
    test_df,
    TEST_DIR,
    x_col='filename',
    y_col=None, # Não há rótulos para os dados de teste
    class_mode=None,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False # Importante manter a ordem para a submissão
)

# Realizando as predições
predict = model.predict(test_generator, steps=np.ceil(nb_samples/BATCH_SIZE))

# Convertendo probabilidades em classes (0 ou 1)
# A classe 'dog' é 1, 'cat' é 0. O sigmoid > 0.5 indica maior probabilidade de ser a classe 1.
test_df['category'] = np.where(predict > 0.5, 1, 0)

# Criando o DataFrame de submissão
submission_df = test_df.copy()
submission_df['id'] = submission_df['filename'].str.split('.').str[0]
submission_df['label'] = submission_df['category']
submission_df.drop(['filename', 'category'], axis=1, inplace=True)

print("\n---| DataFrame de Submissão para Kaggle |---")
print(submission_df.head())

# Gerando o arquivo de submissão
submission_df.to_csv('submission_cnn.csv', index=False)
print("\nArquivo 'submission_cnn.csv' gerado com sucesso!")

### 8. Conclusão

Neste notebook, implementamos um pipeline completo para classificação de imagens com uma Rede Neural Convolucional. Cobrimos desde o pré-processamento dos dados, passando pela definição e treinamento de uma arquitetura de CNN, até a avaliação do desempenho e a geração de um arquivo de predições para o conjunto de teste. Os resultados mostram a eficácia das CNNs para tarefas de visão computacional, e o modelo pode ser ainda mais aprimorado com mais épocas de treinamento, uma arquitetura mais complexa ou técnicas de ajuste fino (fine-tuning).